In [ ]:
import json
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
api_key ='Your API keys'
openai.api_key = api_key

In [ ]:
test_data=pd.read_csv('koc_test_1.csv')
train_data=pd.read_csv('koc_train_1.csv')
train_data = train_data[['new_smile','Observed']]
test_data = test_data[['new_smile','Observed']]

In [ ]:
def smi_tokenizer(smi):
    """
    Tokenize a SMILES molecule or reaction
    """
    import re
    pattern =  "(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\\\|\/|:|~|@|\?|>|\*|\$|\%[0-9]{2}|[0-9])"
    regex = re.compile(pattern)
    tokens = [token for token in regex.findall(smi)]
    #assert smi == ''.join(tokens)
    return ' '.join(tokens)

In [ ]:
def tok(target):
    return ' '.join(list(str(target)))

In [ ]:
train_data['prompt']=train_data['new_smile'].apply(smi_tokenizer)
test_data['prompt']=test_data['new_smile'].apply(smi_tokenizer)
train_data['completion']=train_data['Observed'].apply(tok)
test_data['completion']=test_data['Observed'].apply(tok)
train_data['prompt'] = train_data['prompt'] + " " +  '->'
test_data['prompt'] = test_data['prompt'] + " " +  '->'
train_data['completion'] = train_data['completion'] + " " +  '\n'
test_data['completion'] = test_data['completion'] + " " +  '\n'

In [ ]:
train_data_file = [{'prompt': train_data['prompt'][i], 'completion': train_data['completion'][i] } for i in train_data.index]
file_name = "training_data_koc_babbage.jsonl"
with open(file_name, "w") as output_file:
    for entry in train_data_file:
        json.dump(entry, output_file)
        output_file.write("\n")

In [ ]:
upload_response = openai.File.create(file=open(file_name, "rb"),purpose='fine-tune')
file_id = upload_response.id
fine_tune_response = openai.FineTune.create(training_file=file_id,model='babbage', suffix = 'koc')
if fine_tune_response.fine_tuned_model == None:
    fine_tuned_model = openai.FineTune.retrieve(id=fine_tune_response.id).fine_tuned_model
print(fine_tuned_model)

In [ ]:
y_pred_test = []
for i in test_data.index:
    new_prompt = test_data['prompt'][i]
    answer = openai.Completion.create(model=fine_tuned_model,prompt=new_prompt,max_tokens=100,temperature=0, stop = ['\n'])
    pred_test = float(answer['choices'][0]['text'].replace(' ', ''))
    y_pred_test.append(pred_test)

In [ ]:
y_pred_train = []
for i in train_data.index:
    new_prompt = train_data['prompt'][i]
    answer = openai.Completion.create(model=fine_tuned_model,prompt=new_prompt,max_tokens=100,temperature=0, stop = ['\n'])
    pred_train = float(answer['choices'][0]['text'].replace(' ', ''))
    y_pred_train.append(pred_train)

In [ ]:
y_pred_train=np.array(y_pred_train)
y_pred_test=np.array(y_pred_test)

In [ ]:
y_true_trian = train_data_raw['Observed'].values
y_true_test = test_data_raw['Observed'].values